# Fake News Classificer Using Bidirectional LSTM

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv',engine='python',on_bad_lines='skip')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [5]:
df.shape

(20822, 5)

In [6]:
df = df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
## get the independent features
X=df.drop('label',axis=1)

In [9]:
y=df['label']

In [10]:
## Check whether dataset is balanced or not
y.value_counts()

0                                                   10361
1                                                    7922
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: label, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional # for bi-directional lstm

In [13]:
## vocabulary size
voc_size=5000

In [14]:
messages=X.copy()

In [15]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
messages.reset_index(inplace=True)

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4837, 2545, 3624, 289, 1549, 4920, 3307, 1919, 1850, 4388],
 [4963, 2710, 208, 2849, 431, 1218, 3716],
 [50, 2703, 1931, 2375],
 [3119, 4502, 2406, 3706, 4941, 2025],
 [4135, 431, 372, 394, 591, 2986, 431, 3931, 156, 3975],
 [3829,
  22,
  635,
  4081,
  245,
  3609,
  322,
  80,
  2840,
  3175,
  2138,
  3106,
  688,
  774,
  3716],
 [849, 3984, 2717, 944, 263, 1110, 2207, 4800, 3076, 3463, 15],
 [2793, 3940, 4076, 3604, 3830, 3205, 3609, 658, 3076, 3463, 15],
 [2881, 910, 198, 1192, 3115, 2944, 1532, 726, 3609, 1321],
 [3299, 2552, 3761, 1357, 142, 1366, 1054, 4342],
 [2900, 1097, 4770, 3623, 2902, 4081, 97, 920, 2012, 303, 1908],
 [3706, 1325, 1549, 2944, 3609, 3830],
 [518, 4518, 599, 2333, 2088, 1620, 3720, 1686, 2061],
 [553, 535, 1356, 4245, 1544, 1035, 743, 3076, 3463, 15],
 [3137, 3610, 1615, 3938, 4799, 3076, 3463, 15],
 [3600, 1821, 2100, 1578, 1006, 568, 4408, 2548, 4451, 3849],
 [2040, 4463, 2710],
 [1356, 4937, 1908, 2728, 3609, 661, 1127, 3716],
 [2664, 3394, 208, 4324

# Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 1919, 1850, 4388],
       [   0,    0,    0, ...,  431, 1218, 3716],
       [   0,    0,    0, ..., 2703, 1931, 2375],
       ...,
       [   0,    0,    0, ..., 3076, 3463,   15],
       [   0,    0,    0, ..., 3635, 4750, 4302],
       [   0,    0,    0, ..., 4630,  465, 2404]])

In [23]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100))) # adding a bidirectional lstm rnn layer
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=0)

In [29]:
# convertion of data type
if isinstance(X_train, np.ndarray):
    X_train = pd.DataFrame(X_train)
    
y_train = pd.Series(y_train)

In [30]:
# Reset indices of X_train and y_train to ensure alignment
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [31]:
# Convert pandas series to numeric types since labels are in string format
y_train = pd.to_numeric(y_train, errors='coerce')
y_test = pd.to_numeric(y_test, errors='coerce')

In [35]:
print(np.any(np.isnan(y_test)))
print(np.any(np.isnan(y_train)))

False
False


In [33]:
# finding nan values index
nan_indices = y_train[y_train.isna()].index

In [34]:
# dropping nan values row from y_train and X_train
y_train = y_train.drop(nan_indices)
X_train = X_train.drop(nan_indices)

In [36]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
458/458 [==============================] - 20s 31ms/step - loss: 0.2480 - accuracy: 0.8914 - val_loss: 0.1874 - val_accuracy: 0.9204
Epoch 2/10
458/458 [==============================] - 13s 29ms/step - loss: 0.1391 - accuracy: 0.9450 - val_loss: 0.1860 - val_accuracy: 0.9292
Epoch 3/10
458/458 [==============================] - 13s 28ms/step - loss: 0.1007 - accuracy: 0.9616 - val_loss: 0.2176 - val_accuracy: 0.9210
Epoch 4/10
458/458 [==============================] - 13s 28ms/step - loss: 0.0696 - accuracy: 0.9755 - val_loss: 0.2306 - val_accuracy: 0.9251
Epoch 5/10
458/458 [==============================] - 13s 29ms/step - loss: 0.0479 - accuracy: 0.9839 - val_loss: 0.2504 - val_accuracy: 0.9191
Epoch 6/10
458/458 [==============================] - 13s 29ms/step - loss: 0.0285 - accuracy: 0.9897 - val_loss: 0.3729 - val_accuracy: 0.9221
Epoch 7/10
458/458 [==============================] - 13s 29ms/step - loss: 0.0200 - accuracy: 0.9940 - val_loss: 0.3845 - val_accuracy:

In [37]:
y_pred=model.predict(X_test)

115/115 [==============================] - 2s 10ms/step


In [38]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [39]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[1901,  146],
       [ 189, 1421]], dtype=int64)

In [40]:
print(accuracy_score(y_test,y_pred))

0.9083948591741865
